Library Imports / Display Settings (please update environment display according to your setup)

You may uncomment if you want to visualize other steps

In [ ]:
from robotic import ry
import time
import os
os.environ['DISPLAY'] = 'LAPTOP-8K2E2M8V.local:0'

In [ ]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')
#C.view()
qHome = C.getJointState()

Get Final Pose With KOMO

In [ ]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

#for t in range(q.shape[0]):
#    C.setJointState(q[t])
#    C.view(False, f'waypoint {t}')
#    time.sleep(.1)

qT = komo.getPath()[-1]
C.setJointState(qT)
#C.view(False, "IK solution")

RRT (Rapidly Exploring Random Tree)

In [ ]:
#define a path finding problem
myBool = False
while myBool is False: 
    rrt = ry.PathFinder()
    rrt.setProblem(C, [qHome], [qT])

    ret = rrt.solve()
    myBool = ret.feasible
    path = ret.x

In [ ]:
# display the path
#for t in range(0, path.shape[0]-1):
#    C.setJointState(path[t])
#    C.view()
#    time.sleep(.1)

In [ ]:
del komo
del C
del rrt

In [ ]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')
#C.view()
qHome = C.getJointState()

Trajectory Optimization via KOMO

In [ ]:
C.setJointState(qHome)
komo = ry.KOMO(C, len(path), 1, 2, True)
#komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 1, 1e1)
#komo.addControlObjective([], 2, 1e1)
komo.initWithPath_qOrg(path)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([673], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e1])
komo.addObjective([673], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)


ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.01)

In [ ]:
del komo
del C